# Testing multithread efficiency - 11 Mar 2022

Function for running model speed test:

In [2]:
cd /home/wjm41/symlinks/rds/datasets/Ugis

/rds-d2/user/wjm41/hpc-work/datasets/Ugis


In [3]:
from src.utils import get_device

device = get_device()
device

'cuda'

In [4]:
import argparse
import time

from src.parsing import add_io_args, add_data_args, add_optim_args

def test_model_speed(model, epochs=1, batch_size=16384, minibatch_size=32):
    parser = argparse.ArgumentParser()
    parser = add_io_args(parser)
    parser = add_data_args(parser)
    parser = add_optim_args(parser)

    parser.add_argument('-time_forward_pass', action='store_true',
                        help='if True, will log the time taken for a forward pass a batch.')
    args = parser.parse_args(['-p', 'test/test_data/HIV.csv',
                              '-y_col', 'activity',
                              '-n_epochs', f'{epochs}',
                              '-batch_size', f'{batch_size}',
                              '-minibatch_size', f'{minibatch_size}',
                              #   '-time_forward_pass'
                              ])
    device = get_device()
    start_time = time.perf_counter()
    model(args, device)
    end_time = time.perf_counter()
    print(f'Total time taken for {epochs} epochs: {end_time-start_time:.1f}s')


Running the speed tests for 10 epochs:

TODO - need to benchmark by the number of steps, not simply the epoch run time

In [5]:
%config Application.log_level="INFO"

In [5]:
import sys

from src.train_and_validate import main as featurize_on_the_fly
from src.train_and_validate_collate import main as pre_featurize
from src.train_and_validate_pmap import main as pre_featurize_pmap

print('Testing featurize-on-the-fly model.')
test_model_speed(featurize_on_the_fly, batch_size=16384)
print('Testing pre-featurised model.')
test_model_speed(pre_featurize, batch_size=16384)
print('Testing pre-featurised pmap model.')
test_model_speed(pre_featurize_pmap, batch_size=16384)
# print('Testing pre-featurised multithread model.')
# test_model_speed(
#     pre_featurize_mutithread)


Testing featurize-on-the-fly model.


100%|█████████████████████████████████████| 3.00/3.00 [01:17<00:00, 25.7s/batch]


Total time taken for 1 epochs: 80.3s
Testing pre-featurised model.


100%|█████████████████████████████████████| 3.00/3.00 [00:02<00:00, 1.16batch/s]


Total time taken for 1 epochs: 77.3s
Testing pre-featurised pmap model.


100%|█████████████████████████████████████| 3.00/3.00 [00:03<00:00, 1.10s/batch]


Total time taken for 1 epochs: 33.0s


In [8]:
from src.train_and_validate_multithread import main as pre_featurize_mutithread

print('Testing pre-featurised multithread model.')
test_model_speed(pre_featurize_mutithread, epochs=3, batch_size=41127, minibatch_size=16384)

Testing pre-featurised multithread model.


100%|█████████████████████████████████████| 1.00/1.00 [01:20<00:00, 80.3s/batch]


Number of minibatches = 14,                         Number of molecules = 229376                             (557.7%)
Number of steps taken: 1


100%|█████████████████████████████████████| 1.00/1.00 [01:18<00:00, 78.7s/batch]


Number of minibatches = 16,                         Number of molecules = 262144                             (637.4%)
Number of steps taken: 1


...

100%|█████████████████████████████████████| 1.00/1.00 [01:21<00:00, 81.7s/batch]


Number of minibatches = 14,                         Number of molecules = 229376                             (557.7%)
Number of steps taken: 1


100%|█████████████████████████████████████| 1.00/1.00 [01:13<00:00, 73.6s/batch]

Number of minibatches = 15,                         Number of molecules = 245760                             (597.6%)
Number of steps taken: 1


Total time taken for 10 epochs: 766.7s
